# Retriever Reader Pipeline

We've setup our Elasticsearch instance and are now ready to prepare the remainder of our ODQA pipeline - our *retriever* and *reader*.

First we initialize the connection to our Elasticsearch document store.

In [1]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)

03/16/2022 08:22:01 - INFO - faiss -   Loading faiss.
03/16/2022 08:22:05 - INFO - elasticsearch -   HEAD http://localhost:9200/aurelius [status:200 request:0.014s]
03/16/2022 08:22:05 - INFO - elasticsearch -   GET http://localhost:9200/aurelius [status:200 request:0.003s]
03/16/2022 08:22:05 - INFO - elasticsearch -   PUT http://localhost:9200/aurelius/_mapping [status:200 request:0.020s]
03/16/2022 08:22:05 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.002s]


Then we initialize our retriever and reader models. We will be using [deepset/bert-base-cased-squad2](https://huggingface.co/deepset/bert-base-cased-squad2) as our reader model.

In [2]:
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.reader.farm import FARMReader

retriever = ElasticsearchRetriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

03/16/2022 08:22:24 - INFO - farm.utils -   Using device: CPU 
03/16/2022 08:22:24 - INFO - farm.utils -   Number of GPUs: 0
03/16/2022 08:22:24 - INFO - farm.utils -   Distributed Training: False
03/16/2022 08:22:24 - INFO - farm.utils -   Automatic Mixed Precision: None
03/16/2022 08:22:40 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
03/16/2022 08:22:40 - INFO - farm.utils -   Using device: CPU 
03/16/2022 08:22:40 - INFO - farm.utils -   Number of GPUs: 0
03/16/2022 08:22:40 - INFO - farm.utils -   Distributed Training: False
03/16/2022 08:22:40 - INFO - farm.utils -   Automatic Mixed Precision: None
03/16/2022 08:22:40 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
03/16/2022 08:22:40 - INFO - farm.infer -    0    0    0    0    0    0    0 
03/16/2022 08:22:40 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\
03/16/2022 08:22:40 - INFO - farm.infer -   /'\  / \  /'\  /'\  / \  / \ 

And now we initialize our ODQA pipeline.

In [3]:
from haystack.pipeline import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

Now we can begin asking questions!

In [4]:
qa.run(query='What did your grandfather teach?')

03/16/2022 08:24:08 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.056s]
Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/Caskroom/miniconda/base/envs/ml/lib/python3.8/site-packages/farm/modeling/prediction_head.py:1112: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.78 Batches/s]


{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.1212520599365234,
 'answers': [{'answer': 'good morals and the government of my temper',
   'score': 5.546537399291992,
   'probability': 0.6667044421188743,
   'context': 'From my grandfather Verus I learned good morals and the government of my temper.',
   'offset_start': 36,
   'offset_end': 79,
   'offset_start_in_doc': 36,
   'offset_end_in_doc': 79,
   'document_id': '6477c2fc-04d1-45a3-9eba-6a9b6ee16112',
   'meta': {'source': 'meditations'}},
  {'answer': 'self-government',
   'score': 1.488312005996704,
   'probability': 0.5463760689986955,
   'context': 'From Maximus I learned self-government, and not to be led aside by anything; and cheerfulness in all circumstances, as well as in illness; and a just admixture in the moral character of sweetness and dignity, and to do what was set before me without complaining. I observed that everybody believed that he thought as he spoke, and that in all that he did he never h

In [5]:
qa.run(query='What is the Universe?',
       top_k_reader=3)

03/16/2022 08:25:09 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.015s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.73 Batches/s]


{'query': 'What is the Universe?',
 'no_ans_gap': 9.206109523773193,
 'answers': [{'answer': 'a well-arranged universe',
   'score': 12.657480239868164,
   'probability': 0.8295137489464071,
   'context': 'Either it is a well-arranged universe or a chaos huddled together, but still a universe. But can a certain order subsist in thee, and disorder in the All? And this too when all things are so separated and diffused and sympathetic.',
   'offset_start': 13,
   'offset_end': 37,
   'offset_start_in_doc': 13,
   'offset_end_in_doc': 37,
   'document_id': 'b94d68ec-866e-498a-9a33-1d95e6d4c2ad',
   'meta': {'source': 'meditations'}},
  {'answer': 'the universe loves to make whatever is about to be',
   'score': 10.453649520874023,
   'probability': 0.7869614603311372,
   'context': '"The earth loves the shower"; and "the solemn aether loves": and the universe loves to make whatever is about to be. I say then to the universe, that I love as thou lovest. And is not this too said, that "this 

In [6]:
qa.run(query='What is art?',
       top_k_reader=3)

03/16/2022 08:27:02 - INFO - elasticsearch -   POST http://localhost:9200/aurelius/_search [status:200 request:0.012s]
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.83 Batches/s]


{'query': 'What is art?',
 'no_ans_gap': 4.066242694854736,
 'answers': [{'answer': 'the acts of life',
   'score': 6.920147895812988,
   'probability': 0.7037080926046142,
   'context': 'Let it make no difference to thee whether thou art cold or warm, if thou art doing thy duty; and whether thou art drowsy or satisfied with sleep; and whether ill-spoken of or praised; and whether dying or doing something else. For it is one of the acts of life, this act by which we die: it is sufficient then in this act also to do well what we have in hand.',
   'offset_start': 244,
   'offset_end': 260,
   'offset_start_in_doc': 244,
   'offset_end_in_doc': 260,
   'document_id': '19cdea02-ffff-4fee-8820-98f690a7abc2',
   'meta': {'source': 'meditations'}},
  {'answer': 'To be good',
   'score': 4.7822113037109375,
   'probability': 0.6451474199464995,
   'context': 'What is thy art? To be good. And how is this accomplished well except by general principles, some about the nature of the universe, and

Pretty cool stuff!